# interface: 從協定到 ABC
## 抽象類別代表介面
* 代表 duck typeing 的動態協定，到使介面更明確，並驗證實作一致的抽象基礎類別(ABC)
* duck typeing 協定他是普通介面思維方式是 ABC 法定程式(formality) 與型態檢查


1. 介紹 duck typeing 動態特質
2. 專門討論 ABC 從常見用途當實作介面將他們當成超類別
3. 何時 ABC 會檢查子類別是否符合他所定義的介面，註冊機制如何壤開發者不需要繼承可以宣稱某類別實作一個介面
4. 編寫 ABC 自動識別符合介面不需要繼承或註冊


## 介面與協定
* 每個類別都有介面包含自己實作或是從類別繼承通用屬性包含特殊方法
* 受保護與私有屬性不是介面一部分
* 公有屬性成為物件一部分，因為資料屬性可以改成實作 getter/setter 邏輯，而不會破壞使用一般obj attr用戶端程式碼

```
#範例9-2
class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>
```

```
# 僵
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y
```